In [18]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers,Sequential
from kerastuner.tuners import RandomSearch

In [2]:
print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)

tf.__version__ is 2.3.0
tf.keras.__version__ is: 2.4.0


In [3]:
(X_train, y_train) , (X_test, y_test) = keras.datasets.mnist.load_data()

In [4]:
X_train = X_train / 255
X_test = X_test / 255

In [5]:
X_train[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [6]:
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from kerastuner.tuners import RandomSearch

In [7]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28,28)))
    model.add(layers.Dense(units=hp.Int('units',
                                            min_value=32,
                                            max_value=512,
                                            step=128),
                               activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2,1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [8]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=2,
    executions_per_trial=2,
    directory='project_dir',
    project_name='minist_keras_tunner')

INFO:tensorflow:Reloading Oracle from existing project project_dir\minist_keras_tunner\oracle.json


In [9]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 128, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.0001], 'ordered': True}


In [10]:
tf.__version__

'2.3.0'

In [11]:
print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)


tf.__version__ is 2.3.0
tf.keras.__version__ is: 2.4.0


In [12]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 2 Complete [00h 00m 58s]
val_accuracy: 0.9597499966621399

Best val_accuracy So Far: 0.9597499966621399
Total elapsed time: 00h 01m 29s
INFO:tensorflow:Oracle triggered exit


In [13]:
tuner.results_summary()

Results summary
Results in project_dir\minist_keras_tunner
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
units: 416
learning_rate: 0.01
Score: 0.9597499966621399
Trial summary
Hyperparameters:
units: 160
learning_rate: 0.01
Score: 0.9596000015735626


In [27]:
def build_model2(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28,28)))
    for i in range(hp.Int('layers', 2,6)):
        model.add(layers.Dense(units=hp.Int('units',min_value=50,max_value=100,step=10),activation=hp.Choice('act_'+str(i),['relu','sigmoid'])))
    model.add(tf.keras.layers.Dense(10,activation='softmax'))
    #model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile('adam','sparse_categorical_crossentropy',metrics=['accuracy'])              
    """model.compile(optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2,1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])"""
    return model


In [28]:
tuner2 = RandomSearch(
    build_model2,
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=1,
    directory='project_dir2',
    project_name='minist_keras_tunner2')

In [29]:
tuner2.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 34s]
val_accuracy: 0.9757000207901001

Best val_accuracy So Far: 0.9761000275611877
Total elapsed time: 00h 02m 56s
INFO:tensorflow:Oracle triggered exit


In [34]:
model=tuner2.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 90)                70650     
_________________________________________________________________
dense_1 (Dense)              (None, 90)                8190      
_________________________________________________________________
dense_2 (Dense)              (None, 90)                8190      
_________________________________________________________________
dense_3 (Dense)              (None, 90)                8190      
_________________________________________________________________
dense_4 (Dense)              (None, 90)                8190      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                9